In [1]:
import warnings
import pprint
import skrebate
import imblearn
import time
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve, GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix, roc_curve
from sklearn import metrics
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
work_dir = "./mIHC/test4_CoarseModel/"

## train a coarse model

In [4]:
# read training dataset
test1n3 = pd.read_csv('./mIHC/test1_C_1rest_ready/test1and3_C_1rest_all.tsv.gz', sep="\t")
test1n3

,obj_ID,patient_ID,class,b_class,AreaShape_Area,Intensity_IntegratedIntensityEdge_CD138_rs,Intensity_IntegratedIntensityEdge_CD20_rs,Intensity_IntegratedIntensityEdge_CD27_rs,Intensity_IntegratedIntensityEdge_CD38_rs,Intensity_IntegratedIntensityEdge_CD3_rs,...,Intensity_UpperQuartileIntensity_IDO_rs,Intensity_UpperQuartileIntensity_IGD_rs,Intensity_UpperQuartileIntensity_IL10_rs,Intensity_UpperQuartileIntensity_KI67_rs,Intensity_UpperQuartileIntensity_MECA79_rs,Intensity_UpperQuartileIntensity_PANCK_rs,Intensity_UpperQuartileIntensity_PD1_rs,Intensity_UpperQuartileIntensity_PDL1_rs,Intensity_UpperQuartileIntensity_TBET_rs,Intensity_UpperQuartileIntensity_TCRB_rs
0,4380R6_19491,4380R6,A,others,372,0.356757,0.017751,0.035971,0.520710,1.331429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014634,0.000000,0.047619,0.319797
1,4380R6_32291,4380R6,A,others,310,0.405405,4.834320,2.769784,0.562130,9.222857,...,0.014388,0.004587,0.000000,0.000000,0.000000,0.000000,0.029268,0.000000,0.011905,0.015228
2,4380R6_30964,4380R6,A,others,243,0.264865,7.852071,0.798561,0.189349,4.331429,...,0.000000,0.009174,0.000000,0.000000,0.000000,0.000000,0.019512,0.006667,0.000000,0.010152
3,4380R6_39772,4380R6,A,others,378,2.097297,23.863905,6.007194,0.153846,18.194286,...,0.035971,0.027523,0.020690,0.000000,0.000000,0.004651,0.068293,0.006667,0.065476,0.015228
4,4380R6_29203,4380R6,A,others,340,4.800000,0.254438,2.381295,12.325444,15.577143,...,0.000000,0.009174,0.013793,0.000000,0.000000,0.000000,0.029268,0.006667,0.071429,0.010152
5,4380R6_5313,4380R6,A,others,106,2.000000,3.396450,3.812950,3.053254,2.514286,...,0.179856,0.110092,0.144828,0.173759,0.122024,0.102326,0.112195,0.173333,0.130952,0.116751
6,4380R6_33223,4380R6,A,others,309,0.400000,2.218935,3.043165,0.142012,14.645714,...,0.028777,0.009174,0.006897,0.000000,0.000000,0.000000,0.023171,0.006667,0.041667,0.005076
7,4380R6_27770,4380R6,A,others,398,0.059459,0.005917,0.021583,0.000000,2.440000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019512,0.000000,0.005952,0.000000
8,4380R6_47126,4380R6,A,others,390,0.600000,0.650888,0.848921,0.278107,4.411429,...,0.007194,0.004587,0.006897,0.000000,0.000000,0.000000,0.004878,0.003333,0.023810,0.012690
9,4380R6_7443,4380R6,A,others,168,4.708108,3.171598,4.424460,3.538461,3.257143,...,0.107914,0.091743,0.144828,0.106383,0.101190,0.065116,0.102439,0.160000,0.089286,0.086294


In [7]:
# read test datasets in ROIs
files = os.listdir(work_dir)
test4 = {}
for fl in files:
    if not fl.endswith('.csv'):
        continue
    test4[fl] = pd.read_csv(work_dir+fl, sep=',')
test4

{'SMA30_ROI04.csv':        ImageNumber  ObjectNumber  AreaShape_Area  AreaShape_Perimeter  \
 0                4             1              50               29.070   
 1                4             2              17               14.484   
 2                4             3             356               80.420   
 3                4             4             197               55.108   
 4                4             5              16               13.312   
 5                4             6              38               25.656   
 6                4             7             110               39.452   
 7                4             8             104               37.796   
 8                4             9              48               25.312   
 9                4            10              14               13.656   
 10               4            11              54               27.312   
 11               4            12             126               42.038   
 12               4

In [8]:
#check duplicate
for k, v in test4.items():
    print(k)
    print(v.duplicated().any())

SMA30_ROI04.csv
False
SMA30_ROI05.csv
False
SMA30_ROI07.csv
False
SMA30_ROI06.csv
False
SMA30_ROI02.csv
False
SMA30_ROI03.csv
False
SMA30_ROI01.csv
False
OPTR4501_ROI04.csv
False
OPTR4501_ROI05.csv
False
OPTR4501_ROI01.csv
False
OPTR4501_ROI02.csv
False
OPTR4501_ROI03.csv
False
SMA56_ROI01.csv
False
OPTR4547_ROI03.csv
False
OPTR4547_ROI02.csv
False
SMA56_ROI02.csv
False
OPTR4547_ROI01.csv
False
SMA56_ROI03.csv
False
OPTR4547_ROI05.csv
False
SMA56_ROI06.csv
False
OPTR4547_ROI04.csv
False
SMA56_ROI04.csv
False
SMA56_ROI05.csv
False


In [9]:
# check feature consistence
columns = []
for k, v in test4.items():
    columns.append(v.columns)
for idx in range(1, len(columns)):
    print(set(columns[idx]) - set(columns[idx-1]))

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [10]:
# get training feature list
feature_list = columns[0].values
feature_list = [x for x in feature_list if x.startswith('Area') or x.startswith('Intensity')]
feature_list = [x for x in feature_list if x != 'AreaShape_Perimeter']
feature_list = sorted(feature_list)
feature_list

['AreaShape_Area',
 'Intensity_MeanIntensity_CD138_rs',
 'Intensity_MeanIntensity_CD20_rs',
 'Intensity_MeanIntensity_CD27_rs',
 'Intensity_MeanIntensity_CD38_rs',
 'Intensity_MeanIntensity_CD3_rs',
 'Intensity_MeanIntensity_CD45_rs',
 'Intensity_MeanIntensity_CD4_rs',
 'Intensity_MeanIntensity_CD5_rs',
 'Intensity_MeanIntensity_CD68_rs',
 'Intensity_MeanIntensity_CD8_rs',
 'Intensity_MeanIntensity_DNA_rs',
 'Intensity_MeanIntensity_EOMES_rs',
 'Intensity_MeanIntensity_GZMB_rs',
 'Intensity_MeanIntensity_ICOS_rs',
 'Intensity_MeanIntensity_IDO_rs',
 'Intensity_MeanIntensity_IGD_rs',
 'Intensity_MeanIntensity_IL10_rs',
 'Intensity_MeanIntensity_KI67_rs',
 'Intensity_MeanIntensity_MECA79_rs',
 'Intensity_MeanIntensity_PANCK_rs',
 'Intensity_MeanIntensity_PD1_rs',
 'Intensity_MeanIntensity_PDL1_rs',
 'Intensity_MeanIntensity_TBET_rs',
 'Intensity_MeanIntensity_TCRB_rs']

In [11]:
# create training dataset
training = test1n3[['class'] + feature_list]
training

,class,AreaShape_Area,Intensity_MeanIntensity_CD138_rs,Intensity_MeanIntensity_CD20_rs,Intensity_MeanIntensity_CD27_rs,Intensity_MeanIntensity_CD38_rs,Intensity_MeanIntensity_CD3_rs,Intensity_MeanIntensity_CD45_rs,Intensity_MeanIntensity_CD4_rs,Intensity_MeanIntensity_CD5_rs,...,Intensity_MeanIntensity_IDO_rs,Intensity_MeanIntensity_IGD_rs,Intensity_MeanIntensity_IL10_rs,Intensity_MeanIntensity_KI67_rs,Intensity_MeanIntensity_MECA79_rs,Intensity_MeanIntensity_PANCK_rs,Intensity_MeanIntensity_PD1_rs,Intensity_MeanIntensity_PDL1_rs,Intensity_MeanIntensity_TBET_rs,Intensity_MeanIntensity_TCRB_rs
0,A,372,0.007977,0.013377,0.020848,0.008430,0.132873,0.244517,0.002096,0.001436,...,0.002089,0.003749,0.002410,0.000286,0.000608,0.000988,0.010058,0.001344,0.035250,0.200999
1,A,310,0.008928,0.055621,0.080297,0.003340,0.176940,0.335628,0.101536,0.046926,...,0.007914,0.004543,0.004004,0.003821,0.002035,0.003226,0.018112,0.002237,0.007124,0.006763
2,A,243,0.004627,0.251930,0.009237,0.001437,0.079365,0.262376,0.015089,0.003676,...,0.004293,0.006267,0.008372,0.002101,0.002229,0.001302,0.010840,0.006557,0.004409,0.008063
3,A,378,0.015501,0.324708,0.066385,0.003992,0.232593,0.501781,0.056201,0.017184,...,0.025732,0.018106,0.013647,0.007374,0.001244,0.008761,0.043696,0.005644,0.033777,0.009561
4,A,340,0.073514,0.003411,0.047186,0.212113,0.278622,0.271258,0.012272,0.068313,...,0.002285,0.004628,0.005619,0.000814,0.000053,0.000780,0.017991,0.003216,0.043505,0.006241
5,A,106,0.090260,0.103718,0.098819,0.071229,0.084367,0.087895,0.075921,0.053137,...,0.104384,0.074347,0.089590,0.104643,0.069519,0.063449,0.077266,0.098176,0.078841,0.071066
6,A,309,0.008729,0.018058,0.052665,0.002700,0.290504,0.431558,0.176953,0.102398,...,0.018160,0.006027,0.007432,0.005004,0.000636,0.002679,0.016892,0.006235,0.025100,0.005355
7,A,398,0.002458,0.000491,0.015057,0.000149,0.160660,0.131997,0.054080,0.027746,...,0.000831,0.002098,0.001577,0.000624,0.000823,0.000304,0.010884,0.000988,0.009078,0.000357
8,A,390,0.004380,0.003899,0.007360,0.001548,0.136791,0.307032,0.008745,0.016984,...,0.007010,0.005857,0.013245,0.001964,0.002320,0.001264,0.004878,0.004957,0.012958,0.017480
9,A,168,0.102477,0.069562,0.085260,0.093195,0.072041,0.059848,0.090668,0.057234,...,0.074298,0.063893,0.091626,0.063534,0.068063,0.047508,0.067334,0.112698,0.068169,0.063512


In [12]:
# save the dataset
#training.to_csv(work_dir+'training_test1n3.tsv.gz', sep='\t', index=False, compression='gzip')

In [13]:
# rearrange the test features
predict_features = ['ImageNumber', 'ObjectNumber', 'AreaShape_Perimeter',
                    'Location_Center_X', 'Location_Center_Y', 'Number_Object_Number']\
                    + feature_list
predict_features

['ImageNumber',
 'ObjectNumber',
 'AreaShape_Perimeter',
 'Location_Center_X',
 'Location_Center_Y',
 'Number_Object_Number',
 'AreaShape_Area',
 'Intensity_MeanIntensity_CD138_rs',
 'Intensity_MeanIntensity_CD20_rs',
 'Intensity_MeanIntensity_CD27_rs',
 'Intensity_MeanIntensity_CD38_rs',
 'Intensity_MeanIntensity_CD3_rs',
 'Intensity_MeanIntensity_CD45_rs',
 'Intensity_MeanIntensity_CD4_rs',
 'Intensity_MeanIntensity_CD5_rs',
 'Intensity_MeanIntensity_CD68_rs',
 'Intensity_MeanIntensity_CD8_rs',
 'Intensity_MeanIntensity_DNA_rs',
 'Intensity_MeanIntensity_EOMES_rs',
 'Intensity_MeanIntensity_GZMB_rs',
 'Intensity_MeanIntensity_ICOS_rs',
 'Intensity_MeanIntensity_IDO_rs',
 'Intensity_MeanIntensity_IGD_rs',
 'Intensity_MeanIntensity_IL10_rs',
 'Intensity_MeanIntensity_KI67_rs',
 'Intensity_MeanIntensity_MECA79_rs',
 'Intensity_MeanIntensity_PANCK_rs',
 'Intensity_MeanIntensity_PD1_rs',
 'Intensity_MeanIntensity_PDL1_rs',
 'Intensity_MeanIntensity_TBET_rs',
 'Intensity_MeanIntensity_TCRB

In [14]:
# create datasets containing the testing features
for k, v in test4.items():
    print(k)
    new_df = v[predict_features]
    #new_df.to_csv(work_dir+'ready_'+k+'.gz', sep='\t', index=False, compression='gzip')

SMA30_ROI04.csv
SMA30_ROI05.csv
SMA30_ROI07.csv
SMA30_ROI06.csv
SMA30_ROI02.csv
SMA30_ROI03.csv
SMA30_ROI01.csv
OPTR4501_ROI04.csv
OPTR4501_ROI05.csv
OPTR4501_ROI01.csv
OPTR4501_ROI02.csv
OPTR4501_ROI03.csv
SMA56_ROI01.csv
OPTR4547_ROI03.csv
OPTR4547_ROI02.csv
SMA56_ROI02.csv
OPTR4547_ROI01.csv
SMA56_ROI03.csv
OPTR4547_ROI05.csv
SMA56_ROI06.csv
OPTR4547_ROI04.csv
SMA56_ROI04.csv
SMA56_ROI05.csv


## Results

In [15]:
result_files = os.listdir(work_dir+'predicted')
result_files

['ready_SMA30_ROI01.csv.gz.tabular',
 'Predicted_OPTR4501_ROI01.csv',
 'Predicted_SMA30_ROI05.csv',
 'Predicted_SMA30_ROI04.csv',
 'ready_OPTR4501_ROI02.csv.gz.tabular',
 'Predicted_OPTR4501_ROI02.csv',
 'ready_SMA30_ROI06.csv.gz.tabular',
 'Predicted_SMA30_ROI06.csv',
 'Predicted_SMA30_ROI07.csv',
 'ready_OPTR4501_ROI05.csv.gz.tabular',
 'Predicted_OPTR4501_ROI03.csv',
 'ready_OPTR4501_ROI04.csv.gz.tabular',
 'Predicted_SMA30_ROI03.csv',
 'Predicted_SMA30_ROI02.csv',
 'ready_SMA30_ROI07.csv.gz.tabular',
 'ready_OPTR4501_ROI03.csv.gz.tabular',
 'Predicted_OPTR4501_ROI04.csv',
 'Predicted_SMA30_ROI01.csv',
 'Predicted_OPTR4501_ROI05.csv',
 'ready_OPTR4501_ROI01.csv.gz.tabular',
 'ready_SMA30_ROI02.csv.gz.tabular',
 'ready_SMA30_ROI05.csv.gz.tabular',
 'ready_SMA30_ROI04.csv.gz.tabular',
 'ready_SMA30_ROI03.csv.gz.tabular',
 'ready_OPTR4547_ROI04.csv.gz.tabular',
 'Predicted_OPTR4547_ROI05.csv',
 'ready_SMA56_ROI01.csv.gz.tabular',
 'Predicted_OPTR4547_ROI04.csv',
 'ready_OPTR4547_ROI03.

In [16]:
# Predicted class distribution
for fl in result_files:
    if not fl.endswith('.gz.tabular'):
        continue
    print(fl)
    path = os.path.join(work_dir+'predicted/'+fl)
    df = pd.read_csv(path, sep='\t')
    print(df['Predicted'].value_counts().to_frame('Counts'))

ready_SMA30_ROI01.csv.gz.tabular
   Counts
F   44177
H   21109
G   13050
E    2054
B    1903
A    1246
D     132
C      45
ready_OPTR4501_ROI02.csv.gz.tabular
   Counts
F   21874
H   18104
G   14641
E    5617
D    1770
B    1678
A     592
C      14
ready_SMA30_ROI06.csv.gz.tabular
   Counts
F   22086
G    7039
H    4061
B     498
E     433
A     428
C      22
D      18
ready_OPTR4501_ROI05.csv.gz.tabular
   Counts
F   35050
G   17970
E    2661
A    2092
B    2027
H    1763
D     693
C      47
ready_OPTR4501_ROI04.csv.gz.tabular
   Counts
F   22151
G    6239
H    2195
E    1551
A     920
B     573
D     507
C       4
ready_SMA30_ROI07.csv.gz.tabular
   Counts
F   11084
G    2593
H    2161
E     157
A     109
B     101
D      16
C       3
ready_OPTR4501_ROI03.csv.gz.tabular
   Counts
F   28166
H   10529
G   10177
E    2939
D    1282
B     707
A     302
C       6
ready_OPTR4501_ROI01.csv.gz.tabular
   Counts
F   43663
G   16032
H   14478
E   11260
B    2844
D    2242
A    1701
C      58
r

In [18]:
# save combined files
for fl in result_files:
    if not fl.endswith('.gz.tabular'):
        continue
    print(fl)
    path = os.path.join(work_dir+'predicted/'+fl)
    df = pd.read_csv(path, sep='\t')
    key = fl[6:-11]
    print(key)
    combined_df = test4[key]
    combined_df['Predicted'] = df['Predicted']
    #combined_df.to_csv(work_dir+'predicted/'+'Predicted_'+key, sep=',', index=False)


ready_SMA30_ROI01.csv.gz.tabular
SMA30_ROI01.csv
ready_OPTR4501_ROI02.csv.gz.tabular
OPTR4501_ROI02.csv
ready_SMA30_ROI06.csv.gz.tabular
SMA30_ROI06.csv
ready_OPTR4501_ROI05.csv.gz.tabular
OPTR4501_ROI05.csv
ready_OPTR4501_ROI04.csv.gz.tabular
OPTR4501_ROI04.csv
ready_SMA30_ROI07.csv.gz.tabular
SMA30_ROI07.csv
ready_OPTR4501_ROI03.csv.gz.tabular
OPTR4501_ROI03.csv
ready_OPTR4501_ROI01.csv.gz.tabular
OPTR4501_ROI01.csv
ready_SMA30_ROI02.csv.gz.tabular
SMA30_ROI02.csv
ready_SMA30_ROI05.csv.gz.tabular
SMA30_ROI05.csv
ready_SMA30_ROI04.csv.gz.tabular
SMA30_ROI04.csv
ready_SMA30_ROI03.csv.gz.tabular
SMA30_ROI03.csv
ready_OPTR4547_ROI04.csv.gz.tabular
OPTR4547_ROI04.csv
ready_SMA56_ROI01.csv.gz.tabular
SMA56_ROI01.csv
ready_OPTR4547_ROI03.csv.gz.tabular
OPTR4547_ROI03.csv
ready_SMA56_ROI06.csv.gz.tabular
SMA56_ROI06.csv
ready_OPTR4547_ROI02.csv.gz.tabular
OPTR4547_ROI02.csv
ready_OPTR4547_ROI05.csv.gz.tabular
OPTR4547_ROI05.csv
ready_SMA56_ROI02.csv.gz.tabular
SMA56_ROI02.csv
ready_SMA56_ROI